経営管理でまとめている****年度_登録口座数.xlsxをまとめるデータ

In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\programdata\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install -U openpyxl

  Using cached openpyxl-3.0.5-py2.py3-none-any.whl (242 kB)
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.4
    Uninstalling openpyxl-3.0.4:
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] アクセスが拒否されました。: 'c:\\programdata\\anaconda3\\lib\\site-packages\\openpyxl-3.0.4.dist-info\\direct_url.json'
Consider using the `--user` option or check the permissions.



In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

#from pandasql import sqldf

In [3]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20201103


In [4]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [5]:
#ディレクトリ指定
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202010_スコアカード")

In [7]:
filename = ["2018年度_登録口座数.xlsx","2019年度_登録口座数.xlsx","2020年9月度_登録口座数.xlsx"]

登録口座数

In [8]:
li = []#空のデータフレーム作成

2020年度用

In [9]:
%%time
shop_list = ['心斎橋店','梅田店',"京都店","神戸店","東京店","札幌店","名古屋店",
             "上野店","静岡店","銀座店","高槻店","下関店","高知大丸"]


for f in filename:
    input_file = f
    input_book = pd.ExcelFile(data_dir/ f"{input_file}")
    #各シート名を取得
    input_sheet_name = input_book.sheet_names
    num_sheet = len(input_sheet_name)
    
    for i in range(0,num_sheet-1):#不要なシートが一つあるので読みこまない
        for shop in shop_list:
            df = input_book.parse(input_sheet_name[i])
            
            if f[:5] == "2020年":
                df = df.iloc[[47,62,83,104,125,146,174,202,209],[1,4,5,6,7,8,9,10,11,12,13,14,15,17]]#本月末口座数の行と店舗名の列だけ抽出
                df = df.rename(columns={"Unnamed: 1":"card",
                                        "Unnamed: 4":"心斎橋店",
                                        "Unnamed: 5":"梅田店",
                                        "Unnamed: 6":"京都店",
                                        "Unnamed: 7":"神戸店",
                                        "Unnamed: 8":"東京店",
                                        "Unnamed: 9":"札幌店",
                                        "Unnamed: 10":"名古屋店",
                                        "Unnamed: 11":"上野店",
                                        "Unnamed: 12":"静岡店",
                                        "Unnamed: 13":"銀座店",
                                        "Unnamed: 14":"高槻店",
                                        "Unnamed: 15":"下関店",
                                        "Unnamed: 17":"高知大丸"})
            
                df = df.drop(df.index[0])#値の入ってない最初の行を削除
                df = df[['card',shop]].rename(columns={shop:'登録口座数'})
                df['DM店名'] = shop
                #余計な文字削除
                df['card'] = df['card'].str.replace("\n",'')
                df['card'] = df['card'].str.replace("大丸松坂屋カード",'')
                df['card'] = df['card'].str.replace("大丸松坂屋",'')

                #ブランド分け
                df.loc[df['card'].str.contains("ＶＩＳＡ"), 'brand'] = 'VISA'
                #df.loc[df['card'] == "ＶＩＳＡ", 'brand'] = 'VISA'
                #df.loc[df['card'] == "ゴールドＶＩＳＡ", 'brand'] = 'VISA'
                #df.loc[df['card'] == "お得意様ゴールド ＶＩＳＡ", 'brand'] = 'VISA'

                df.loc[df['card'].str.contains("Ｍａｓｔｅｒ"), 'brand'] = 'Master'
                #df.loc[df['card'] == "Ｍａｓｔｅｒ", 'brand'] = 'Master'
                #df.loc[df['card'] == "ゴールドＭａｓｔｅｒ", 'brand'] = 'Master'
                #df.loc[df['card'] == "お得意様ゴールド Ｍａｓｔｅｒ", 'brand'] = 'Master'

                df.loc[df['card'].str.contains("ハウス"), 'brand'] = 'ハウス'
                #df.loc[df['card'] == "ゴールドハウス", 'brand'] = 'ハウス'

                df.loc[df['card'] .str.contains("臨時"), 'brand'] = 'VISA'

                df.loc[df['card'].str.match('ＶＩＳＡ'), 'card'] = '一般'
                df.loc[df['card'].str.match('Ｍａｓｔｅｒ'), 'card'] = '一般'

                df.loc[df['card'].str.contains("臨時"), 'card'] = '臨時カード'
                df.loc[df['card'].str.contains("ハウス"), 'card'] = 'ハウスゴールド'

                df['card'] = df['card'].str.replace("ＶＩＳＡ",'')
                df['card'] = df['card'].str.replace("Ｍａｓｔｅｒ",'')
                #年月の列を作成
                df['yymm'] = input_sheet_name[i]
                df['year'] = df['yymm'].str[:4]
                df['month'] = df['yymm'].str[5:]
                df = df.drop(columns={'yymm'})

                li.append(df)
                
                #df_all = pd.concat(li, axis=0, ignore_index=True)

#df_all = df_all.reindex(columns = ['year','month','brand','card','shop_name','no_of_account'])

Wall time: 10.2 s


2018-2019年度用

In [11]:
%%time
#2018-2019年度　※若干レイアウトが異なるのと、下関大丸あり
shop_list = ['心斎橋店','梅田店',"京都店","神戸店","東京店","札幌店","名古屋店",
             "上野店","静岡店","銀座店","高槻店","高知大丸","下関大丸"]

for f in filename:
    input_file = f
    input_book = pd.ExcelFile(data_dir/ f"{input_file}")
    #各シート名を取得
    input_sheet_name = input_book.sheet_names
    num_sheet = len(input_sheet_name)
    
    for i in range(0,num_sheet-1):#不要なシートが一つあるので読みこまない
        for shop in shop_list:
            df = input_book.parse(input_sheet_name[i])
            
            if f[:5] == "2018年" or f[:5] == "2019年":
                df = df.iloc[[47,62,83,104,125,146,174,202,209],[1,3,4,5,6,7,8,10,11,12,13,14,17,18]]#本月末口座数の行と店舗名の列だけ抽出
                df = df.rename(columns={"Unnamed: 1":"card",
                                        "Unnamed: 3":"心斎橋店",
                                        "Unnamed: 4":"梅田店",
                                        "Unnamed: 5":"京都店",
                                        "Unnamed: 6":"神戸店",
                                        "Unnamed: 7":"東京店",
                                        "Unnamed: 8":"札幌店",
                                        "Unnamed: 10":"名古屋店",
                                        "Unnamed: 11":"上野店",
                                        "Unnamed: 12":"静岡店",
                                        "Unnamed: 13":"銀座店",
                                        "Unnamed: 14":"高槻店",
                                        "Unnamed: 17":"高知大丸",
                                        "Unnamed: 18":"下関大丸"})
            
                df = df.drop(df.index[0])#値の入ってない最初の行を削除
                df = df[['card',shop]].rename(columns={shop:'登録口座数'})
                df['DM店名'] = shop
                #余計な文字削除
                df['card'] = df['card'].str.replace("\n",'')
                df['card'] = df['card'].str.replace("大丸松坂屋カード",'')
                df['card'] = df['card'].str.replace("大丸松坂屋",'')

                #ブランド分け
                df.loc[df['card'].str.contains("ＶＩＳＡ"), 'brand'] = 'VISA'
                #df.loc[df['card'] == "ＶＩＳＡ", 'brand'] = 'VISA'
                #df.loc[df['card'] == "ゴールドＶＩＳＡ", 'brand'] = 'VISA'
                #df.loc[df['card'] == "お得意様ゴールド ＶＩＳＡ", 'brand'] = 'VISA'

                df.loc[df['card'].str.contains("Ｍａｓｔｅｒ"), 'brand'] = 'Master'
                #df.loc[df['card'] == "Ｍａｓｔｅｒ", 'brand'] = 'Master'
                #df.loc[df['card'] == "ゴールドＭａｓｔｅｒ", 'brand'] = 'Master'
                #df.loc[df['card'] == "お得意様ゴールド Ｍａｓｔｅｒ", 'brand'] = 'Master'

                df.loc[df['card'].str.contains("ハウス"), 'brand'] = 'ハウス'
                #df.loc[df['card'] == "ゴールドハウス", 'brand'] = 'ハウス'

                df.loc[df['card'] .str.contains("臨時"), 'brand'] = 'VISA'

                df.loc[df['card'].str.match('ＶＩＳＡ'), 'card'] = '一般'
                df.loc[df['card'].str.match('Ｍａｓｔｅｒ'), 'card'] = '一般'

                df.loc[df['card'].str.contains("臨時"), 'card'] = '臨時カード'
                df.loc[df['card'].str.contains("ハウス"), 'card'] = 'ハウスゴールド'

                df['card'] = df['card'].str.replace("ＶＩＳＡ",'')
                df['card'] = df['card'].str.replace("Ｍａｓｔｅｒ",'')
                #年月の列を作成
                df['yymm'] = input_sheet_name[i]
                df['year'] = df['yymm'].str[:4]
                df['month'] = df['yymm'].str[5:]
                df = df.drop(columns={'yymm'})

                li.append(df)
                
                #df_all = pd.concat(li, axis=0, ignore_index=True)

#df_all = df_all.reindex(columns = ['year','month','brand','card','shop_name','no_of_account'])

Wall time: 13.2 s


In [12]:
df_all = pd.concat(li, axis=0, ignore_index=True)
df_all = df_all.reindex(columns = ['year','month','brand','card','DM店名','登録口座数'])
df_all = df_all.sort_values(['year','month','brand','card'])

In [13]:
df_all.head()

,year,month,brand,card,DM店名,登録口座数
734,2018,03,Master,お得意様ゴールド,心斎橋店,2673
742,2018,03,Master,お得意様ゴールド,梅田店,986
750,2018,03,Master,お得意様ゴールド,京都店,2800
758,2018,03,Master,お得意様ゴールド,神戸店,2652
766,2018,03,Master,お得意様ゴールド,東京店,1810


In [14]:
df_all.groupby(['year','month'])['登録口座数'].sum()

year  month
2018  03       1600919
      04       1601225
      05       1605135
      06       1605827
      07       1602995
      08       1603932
      09       1602567
      10       1603361
      11       1605849
      12       1607310
2019  01       1605212
      02       1603680
      03       1597596
      04       1592894
      05       1590671
      06       1583283
      07       1584394
      08       1580746
      09       1577534
      10       1573232
      11       1570588
      12       1568847
2020  01       1567480
      02       1564765
      03       1558686
      04       1549951
      05       1541556
      06       1532504
      07       1525040
      08       1516006
      09       1497580
Name: 登録口座数, dtype: int64

Excelアウトプット

In [16]:
df_all.to_excel(data_dir /r"Output"/f"【集計】登録口座数_{today_yyyymmdd}.xlsx"
                ,sheet_name ='raw_登録口座数',index=None,header=True,encoding='utf-8-sig')

カード発行枚数

In [17]:
li = []#空のデータフレーム作成

2020年度用

In [18]:
%%time
shop_list = ['心斎橋店','梅田店',"京都店","神戸店","東京店","札幌店","名古屋店",
             "上野店","静岡店","銀座店","高槻店","下関店","高知大丸"]


for f in filename:
    input_file = f
    input_book = pd.ExcelFile(data_dir/ f"{input_file}")
    #各シート名を取得
    input_sheet_name = input_book.sheet_names
    num_sheet = len(input_sheet_name)
    
    for i in range(0,num_sheet-1):#不要なシートが一つあるので読みこまない
        for shop in shop_list:
            df = input_book.parse(input_sheet_name[i])
            
            if f[:5] == "2020年":
                df = df.iloc[[242,257,278,299,320,341,369,397,404],[1,4,5,6,7,8,9,10,11,12,13,14,15,17]]#本月末口座数の行と店舗名の列だけ抽出
                df = df.rename(columns={"Unnamed: 1":"card",
                                        "Unnamed: 4":"心斎橋店",
                                        "Unnamed: 5":"梅田店",
                                        "Unnamed: 6":"京都店",
                                        "Unnamed: 7":"神戸店",
                                        "Unnamed: 8":"東京店",
                                        "Unnamed: 9":"札幌店",
                                        "Unnamed: 10":"名古屋店",
                                        "Unnamed: 11":"上野店",
                                        "Unnamed: 12":"静岡店",
                                        "Unnamed: 13":"銀座店",
                                        "Unnamed: 14":"高槻店",
                                        "Unnamed: 15":"下関店",
                                        "Unnamed: 17":"高知大丸"})
            
                df = df.drop(df.index[0])#値の入ってない最初の行を削除
                df = df[['card',shop]].rename(columns={shop:'発行枚数'})
                df['DM店名'] = shop
                #余計な文字削除
                df['card'] = df['card'].str.replace("\n",'')
                df['card'] = df['card'].str.replace("大丸松坂屋カード",'')
                df['card'] = df['card'].str.replace("大丸松坂屋",'')

                #ブランド分け
                df.loc[df['card'].str.contains("ＶＩＳＡ"), 'brand'] = 'VISA'
                #df.loc[df['card'] == "ＶＩＳＡ", 'brand'] = 'VISA'
                #df.loc[df['card'] == "ゴールドＶＩＳＡ", 'brand'] = 'VISA'
                #df.loc[df['card'] == "お得意様ゴールド ＶＩＳＡ", 'brand'] = 'VISA'

                df.loc[df['card'].str.contains("Ｍａｓｔｅｒ"), 'brand'] = 'Master'
                #df.loc[df['card'] == "Ｍａｓｔｅｒ", 'brand'] = 'Master'
                #df.loc[df['card'] == "ゴールドＭａｓｔｅｒ", 'brand'] = 'Master'
                #df.loc[df['card'] == "お得意様ゴールド Ｍａｓｔｅｒ", 'brand'] = 'Master'

                df.loc[df['card'].str.contains("ハウス"), 'brand'] = 'ハウス'
                #df.loc[df['card'] == "ゴールドハウス", 'brand'] = 'ハウス'

                df.loc[df['card'] .str.contains("臨時"), 'brand'] = 'VISA'

                df.loc[df['card'].str.match('ＶＩＳＡ'), 'card'] = '一般'
                df.loc[df['card'].str.match('Ｍａｓｔｅｒ'), 'card'] = '一般'

                df.loc[df['card'].str.contains("臨時"), 'card'] = '臨時カード'
                df.loc[df['card'].str.contains("ハウス"), 'card'] = 'ハウスゴールド'

                df['card'] = df['card'].str.replace("ＶＩＳＡ",'')
                df['card'] = df['card'].str.replace("Ｍａｓｔｅｒ",'')
                #年月の列を作成
                df['yymm'] = input_sheet_name[i]
                df['year'] = df['yymm'].str[:4]
                df['month'] = df['yymm'].str[5:]
                df = df.drop(columns={'yymm'})

                li.append(df)
                
                #df_all = pd.concat(li, axis=0, ignore_index=True)

#df_all = df_all.reindex(columns = ['year','month','brand','card','shop_name','no_of_account'])

Wall time: 10.5 s


2018-2019年度用

In [19]:
%%time
#2018-2019年度　※若干レイアウトが異なるのと、下関大丸あり
shop_list = ['心斎橋店','梅田店',"京都店","神戸店","東京店","札幌店","名古屋店",
             "上野店","静岡店","銀座店","高槻店","高知大丸","下関大丸"]

for f in filename:
    input_file = f
    input_book = pd.ExcelFile(data_dir/ f"{input_file}")
    #各シート名を取得
    input_sheet_name = input_book.sheet_names
    num_sheet = len(input_sheet_name)
    
    for i in range(0,num_sheet-1):#不要なシートが一つあるので読みこまない
        for shop in shop_list:
            df = input_book.parse(input_sheet_name[i])
            
            if f[:5] == "2018年" or f[:5] == "2019年":
                df = df.iloc[[243,258,279,300,321,342,370,398,405],[1,3,4,5,6,7,8,10,11,12,13,14,17,18]]#本月末口座数の行と店舗名の列だけ抽出
                df = df.rename(columns={"Unnamed: 1":"card",
                                        "Unnamed: 3":"心斎橋店",
                                        "Unnamed: 4":"梅田店",
                                        "Unnamed: 5":"京都店",
                                        "Unnamed: 6":"神戸店",
                                        "Unnamed: 7":"東京店",
                                        "Unnamed: 8":"札幌店",
                                        "Unnamed: 10":"名古屋店",
                                        "Unnamed: 11":"上野店",
                                        "Unnamed: 12":"静岡店",
                                        "Unnamed: 13":"銀座店",
                                        "Unnamed: 14":"高槻店",
                                        "Unnamed: 17":"高知大丸",
                                        "Unnamed: 18":"下関大丸"})
            
                df = df.drop(df.index[0])#値の入ってない最初の行を削除
                df = df[['card',shop]].rename(columns={shop:'発行枚数'})
                df['DM店名'] = shop
                #余計な文字削除
                df['card'] = df['card'].str.replace("\n",'')
                df['card'] = df['card'].str.replace("大丸松坂屋カード",'')
                df['card'] = df['card'].str.replace("大丸松坂屋",'')

                #ブランド分け
                df.loc[df['card'].str.contains("ＶＩＳＡ"), 'brand'] = 'VISA'
                #df.loc[df['card'] == "ＶＩＳＡ", 'brand'] = 'VISA'
                #df.loc[df['card'] == "ゴールドＶＩＳＡ", 'brand'] = 'VISA'
                #df.loc[df['card'] == "お得意様ゴールド ＶＩＳＡ", 'brand'] = 'VISA'

                df.loc[df['card'].str.contains("Ｍａｓｔｅｒ"), 'brand'] = 'Master'
                #df.loc[df['card'] == "Ｍａｓｔｅｒ", 'brand'] = 'Master'
                #df.loc[df['card'] == "ゴールドＭａｓｔｅｒ", 'brand'] = 'Master'
                #df.loc[df['card'] == "お得意様ゴールド Ｍａｓｔｅｒ", 'brand'] = 'Master'

                df.loc[df['card'].str.contains("ハウス"), 'brand'] = 'ハウス'
                #df.loc[df['card'] == "ゴールドハウス", 'brand'] = 'ハウス'

                df.loc[df['card'] .str.contains("臨時"), 'brand'] = 'VISA'

                df.loc[df['card'].str.match('ＶＩＳＡ'), 'card'] = '一般'
                df.loc[df['card'].str.match('Ｍａｓｔｅｒ'), 'card'] = '一般'

                df.loc[df['card'].str.contains("臨時"), 'card'] = '臨時カード'
                df.loc[df['card'].str.contains("ハウス"), 'card'] = 'ハウスゴールド'

                df['card'] = df['card'].str.replace("ＶＩＳＡ",'')
                df['card'] = df['card'].str.replace("Ｍａｓｔｅｒ",'')
                #年月の列を作成
                df['yymm'] = input_sheet_name[i]
                df['year'] = df['yymm'].str[:4]
                df['month'] = df['yymm'].str[5:]
                df = df.drop(columns={'yymm'})

                li.append(df)
                
                #df_all = pd.concat(li, axis=0, ignore_index=True)

#df_all = df_all.reindex(columns = ['year','month','brand','card','shop_name','no_of_account'])

Wall time: 14 s


In [20]:
df_all = pd.concat(li, axis=0, ignore_index=True)
df_all = df_all.reindex(columns = ['year','month','brand','card','DM店名','発行枚数'])
df_all = df_all.sort_values(['year','month','brand','card'])

In [21]:
df_all.head()

,year,month,brand,card,DM店名,発行枚数
734,2018,03,Master,お得意様ゴールド,心斎橋店,5491
742,2018,03,Master,お得意様ゴールド,梅田店,2072
750,2018,03,Master,お得意様ゴールド,京都店,5914
758,2018,03,Master,お得意様ゴールド,神戸店,5648
766,2018,03,Master,お得意様ゴールド,東京店,3294


In [22]:
df_all.groupby(['year','month'])['発行枚数'].sum()

year  month
2018  03       1924604
      04       1924929
      05       1929594
      06       1930536
      07       1927129
      08       1928291
      09       1926448
      10       1927458
      11       1930331
      12       1932251
2019  01       1930423
      02       1929135
      03       1923056
      04       1918398
      05       1916785
      06       1909111
      07       1910873
      08       1907233
      09       1903764
      10       1899652
      11       1897632
      12       1896339
2020  01       1895775
      02       1893446
      03       1887630
      04       1878411
      05       1869380
      06       1860075
      07       1852577
      08       1843074
      09       1821411
Name: 発行枚数, dtype: int64

Excelアウトプット

In [23]:
with pd.ExcelWriter(data_dir /r"Output"/f"【集計】登録口座数_{today_yyyymmdd}.xlsx", mode='a',engine='openpyxl') as writer:
    df_all.to_excel(writer, sheet_name='raw_カード発行枚数',index=None,header=True,encoding='utf-8-sig')